In [313]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go

# output to notebook
init_notebook_mode(True)

## Generating the Data and Visualizing It

The data is basically random gaussian noise, combined with a logarthmic trend, and a cosine-waveform of seasonality with a period of 12 units. In other words, you can think of this data as representing some time series over one hundred years with yearly seasonality.

In [441]:
size = 1200
x = np.arange(size)
com1 = np.random.rand(size)*.5
com2 = np.log(np.linspace(1,2,size))
com3 = np.array([np.cos(2*np.pi*np.arange(12)/12)*.5 for i in range(100)]).reshape(size,)
ts = com1+com2+com3

trace = go.Scatter(
    x=x,
    y=ts
)

fig = [trace]
iplot(fig)

## Functions

The first is the original function from Gregory Trubetskoy's blog.  I use this as a point of comparison.  Our functions vary a bit in terms of functionality.  His returns the original data that's been smoothed as well the next step prediction.  Mine only returns the next prediction.

To compare the effectiveness of vectorized code and numba, I wrote a for-loop implementation as well.  Lastly, I created a wrapper for these functions called `single_exp_smooth`.  These functions are not nested because numba behaves strangely when nested.

In [ ]:
# https://grisha.org/blog/2016/01/29/triple-exponential-smoothing-forecasting/ 
# the final value of the output is equivalent to the SES prediction.
def exp_smooth(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha*series[n] + (1 - alpha)*result[n-1])
    return result

In [447]:
def ses(series,a):
    pred = 0
    for j in range(series.shape[0]):
        pred += a * series[length-(j+1)] * ((1-a) **j)
    return pred

def ses_b(series, a):
    length = series.shape[0]
    exp_ = (1-a) ** np.arange(length-1, -1, -1)
    return a * np.dot(series, exp_)

@jit(nopython=True)
def ses_nb(series, a):
    length = series.shape[0]
    exp_ = np.empty(length)
    
    exp_[length-1] = 1
    a_inv = 1 - a
    for i in np.arange(1,length): # optimized for numba
        exp_[(length-1)-i] = exp_[length-i]*a_inv
    #exp_ = (1-a) ** np.arange(length-1, -1, -1) # brodcasting
    return a * np.dot(series, exp_)

def single_exp_smooth(series, a, method='default'):
    ''' Single Exponential Smoothing with For Loop
    Input: Numpy array, alpha, number of predictions
    Output: Original array with steps number of predictions
    Does not take advantage of vectorized code, naive implementation.
    Notice that this implementation requires two loops.
    Assumes that elements at end of array (on the right) are most recent.
    [Beginning, ..., End]
    '''    
    if 'ndarray' not in str(type(series)):
        print('Please use a numpy array')

    elif method == 'default':
        return ses(series,a)
    
    elif method == 'broadcast':
        return ses_b(series,a)
    
    elif method == 'numba':
        return ses_nb(series,a)

In [448]:
%timeit exp_smooth(ts, .5)

448 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [449]:
%timeit single_exp_smooth(ts, .5, 'default')

511 µs ± 5.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [450]:
%timeit single_exp_smooth(ts, .5, 'broadcast')

19.7 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [451]:
%timeit single_exp_smooth(ts, .5, 'numba')

7.8 µs ± 55.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Broadcasting Exponents versus Loop: Why the Loop Wins

I observed some unusual behavior with the most important being:

* Numba actually slowed down exponential broadcasting
* Numba sped up a simple dependent for-loop CONSIDERABLY

In the first function, I created a matrix representing all of the product operations equivalent to exponentiation.  I figued this was the worst case scenario of what was happening under the hood.

In the second function, I created a loop that takes the previous step and multiples it by $(1 - \alpha)$ (which in our case is simply .5 for illustrative purposes).

In the third function, I complete the same task but in a broadcast-y way.

Finally, I repeat the second function, but without numba to speed up the loop.


In [434]:
def nb_test():
    ind = np.triu_indices(1000,m=1000)
    arr = np.full((1000,1000), .5)
    arr[ind] = 1
    return np.prod(arr,axis=1)

@jit(nopython=True)
def nb_test2():
    out = np.empty(1000)
    out[999] = 1
    for i in np.arange(998,-1,-1):
        out[i] = out[i+1] * .5
    return out

def nb_test3():
    out = np.empty(1000)
    out[999] = 1
    for i in np.arange(998,-1,-1):
        out[i] = out[i+1] * .5
    return out

@jit(nopython=True)
def nb_test4():
    return .5 ** np.arange(999,-1,-1)

def nb_test5():
    return .5 ** np.arange(999,-1,-1)

In [435]:
# vector product of a matrix
%timeit nb_test()

9.76 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [436]:
# dependent loop with numba
%timeit nb_test2()

3.34 µs ± 30 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [437]:
# dependent loop without numba
%timeit nb_test3()

508 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [438]:
# broadcast exponent with numba
%timeit nb_test4()

53 µs ± 792 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [439]:
# broadcast exponent without numba
%timeit nb_test5()

12.5 µs ± 291 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
